## Imports

In [2]:
import sys, os, re, json, hashlib, shutil, glob, random, zipfile, yaml
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import Image, UnidentifiedImageError
import hashlib
import imagehash
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit

## Cleaning: Dataset 1

In [3]:
def _ensure(pkg, import_name=None):
    import importlib
    try:
        importlib.import_module(import_name or pkg)
    except ImportError:
        !pip -q install {pkg}
        importlib.import_module(import_name or pkg)

_ensure("tqdm")
_ensure("pillow", "PIL")
_ensure("imagehash") 

PREFERRED_DATASET_NAME = "dataset1"   
SPLITS_EXPECTED = {"train", "valid", "test"}  
OUT_ROOT = Path("/kaggle/working/clean_dataset1")  
LOG_ROOT = Path("/kaggle/working/logs_dataset1")
OUT_ROOT.mkdir(parents=True, exist_ok=True)
LOG_ROOT.mkdir(parents=True, exist_ok=True)

# Label normalization
LABEL_ALIASES = {
    "healthy": "healthy",
    "normal": "healthy",
    "sick": "ill",
    "ill": "ill",
    "bacterial_dermatosis": "bacterial_dermatosis",
    "bacterial": "bacterial_dermatosis",
    "fungal_infection": "fungal_infection",
    "fungal": "fungal_infection",
    "ear_mites": "ear_mites",
    "otitis": "ear_mites",
    "dental_disease": "dental_disease",
    "teeth_disease": "dental_disease",
    "paw_injury": "paw_injury",
    "paw_disease": "paw_injury",
}

# Extracting a label from a filename when no class folder exists.
FILENAME_LABEL_REGEX = re.compile(r"^[A-Za-z]+")

def find_dataset_root(preferred_name=None):
    input_root = Path("/kaggle/input")
    candidates = [p for p in input_root.iterdir() if p.is_dir()]
    if preferred_name:
        for p in candidates:
            if p.name.lower() == preferred_name.lower():
                return p
    # else try to find one that has the expected splits
    for p in candidates:
        child_dirs = {c.name.lower() for c in p.iterdir() if c.is_dir()}
        if SPLITS_EXPECTED.issubset(child_dirs):
            return p
    return candidates[0] if candidates else None

DATASET_ROOT = find_dataset_root(PREFERRED_DATASET_NAME)
assert DATASET_ROOT is not None, "No dataset found under /kaggle/input."

In [4]:
def normalize_label(raw: str) -> str | None:
    if raw is None:
        return None
    key = raw.strip().lower().replace(" ", "_").replace("-", "_")
    if key in LABEL_ALIASES:
        return LABEL_ALIASES[key]
    return key if key else None

def label_from_filename(name: str) -> str | None:
    stem = Path(name).stem
    m = FILENAME_LABEL_REGEX.match(stem)
    if not m:
        return None
    return normalize_label(m.group(0))

def walk_split(split_dir: Path, split_name: str) -> list[dict]:
    rows = []
    class_dirs = [d for d in split_dir.iterdir() if d.is_dir()]
    files_at_root = [f for f in split_dir.iterdir() if f.is_file()]

    if class_dirs:
        for cdir in class_dirs:
            for img in cdir.rglob("*"):
                if img.is_file():
                    rows.append({
                        "split": split_name,
                        "src_path": str(img),
                        "parent_dir": cdir.name,
                        "label_dir": normalize_label(cdir.name),
                        "label_file": None,
                        "label_source": "dir",
                        "filename": img.name,
                    })
    for img in files_at_root:
        if img.is_file():
            lf = label_from_filename(img.name)
            rows.append({
                "split": split_name,
                "src_path": str(img),
                "parent_dir": None,
                "label_dir": None,
                "label_file": lf,
                "label_source": "file" if lf else None,
                "filename": img.name,
            })
    return rows

def build_inventory(dataset_root: Path) -> pd.DataFrame:
    rows = []
    for split in sorted(SPLITS_EXPECTED):
        split_dir = dataset_root / split
        if not split_dir.exists():
            print(f"Missing split folder: {split_dir}")
            continue
        rows.extend(walk_split(split_dir, split))
    df = pd.DataFrame(rows)
    df["label_raw"] = np.where(df["label_dir"].notna(), df["label_dir"], df["label_file"])
    df["label"] = df["label_raw"]
    df["has_label"] = df["label"].notna()
    return df

inv_df = build_inventory(DATASET_ROOT)

inv_csv = LOG_ROOT / "dataset1_inventory_raw.csv"
inv_df.to_csv(inv_csv, index=False)
print(f"Inventory rows: {len(inv_df):,}  -> saved to {inv_csv}")

def counts_table(df):
    by_split = df.groupby("split").size().rename("images").reset_index()
    by_label = df.groupby(["split", "label"]).size().rename("images").reset_index().sort_values(["split","images"], ascending=[True, False])
    return by_split, by_label

by_split, by_label = counts_table(inv_df)
print("\n--- Images by split (raw) ---")
display(by_split)

print("\n--- Images by split & label (raw; label can be NaN) ---")
display(by_label.head(30)) 

n_unlabeled = (~inv_df["has_label"]).sum()
print(f"\nUnlabeled images detected: {n_unlabeled:,}")

if n_unlabeled:
    print("\nSample unlabeled filenames:")
    display(inv_df.loc[~inv_df["has_label"], ["split","src_path","filename"]].head(10))

Inventory rows: 523  -> saved to /kaggle/working/logs_dataset1/dataset1_inventory_raw.csv

--- Images by split (raw) ---


split  images
0   test      21
1  train     481
2  valid      21


--- Images by split & label (raw; label can be NaN) ---


split label  images
0   test     d      20
1  train     d     480
2  valid     d      20


Unlabeled images detected: 3

Sample unlabeled filenames:


split                                   src_path      filename
19    test   /kaggle/input/dataset1/test/_classes.csv  _classes.csv
417  train  /kaggle/input/dataset1/train/_classes.csv  _classes.csv
517  valid  /kaggle/input/dataset1/valid/_classes.csv  _classes.csv

In [5]:
def load_labels_from_csv_singleclass(dataset_root: Path):
    csv_paths = list(dataset_root.rglob("_classes.csv"))
    print(f"Found {len(csv_paths)} _classes.csv files.")
    all_labels = []
    for csv in csv_paths:
        df = pd.read_csv(csv)
        df.columns = [c.strip().lower() for c in df.columns]
        if "filename" not in df.columns:
            print(f"Missing 'filename' column in {csv}")
            continue

        # Identify class columns
        class_cols = [c for c in df.columns if c != "filename"]
        if not class_cols:
            print(f"No class columns in {csv}")
            continue

        # For dataset 1, it's a single column 'healthy' with 1's
        # Generalize: melt any class cols and take rows with val==1
        melted = df.melt(id_vars="filename", value_vars=class_cols,
                         var_name="label", value_name="val")
        
        pos = melted[melted["val"].astype(str).isin(["1", "True", "true"]) | (melted["val"] == 1)].copy()

        if pos.empty and len(class_cols) == 1:
            pos = df.copy()
            pos["label"] = class_cols[0]
            pos = pos[["filename", "label"]]

        else:
            pos = pos[["filename", "label"]]

        pos["label"] = pos["label"].str.strip().str.lower().str.replace(" ", "_").str.replace("-", "_")
        all_labels.append(pos)

    if not all_labels:
        return pd.DataFrame(columns=["filename", "label"])

    labels_df = pd.concat(all_labels, ignore_index=True).drop_duplicates()
    return labels_df

labels_df = load_labels_from_csv_singleclass(DATASET_ROOT)
print(f"Parsed label rows from CSVs: {len(labels_df)}")
display(labels_df.head())

# Merge with previous inventory inv_df, preferring CSV labels
merged_df = inv_df.copy()
merged_df["filename_lower"] = merged_df["filename"].str.lower()
labels_df["filename_lower"] = labels_df["filename"].str.lower()

merged_df = merged_df.merge(labels_df[["filename_lower","label"]],
                            on="filename_lower", how="left", suffixes=("", "_csv"))

merged_df["label"] = merged_df["label"].combine_first(merged_df["label_csv"])
merged_df["has_label"] = merged_df["label"].notna()
merged_df = merged_df.drop(columns=["filename_lower","label_csv"])

merged_path = LOG_ROOT / "dataset1_inventory_with_csv_labels.csv"
merged_df.to_csv(merged_path, index=False)

# summary to help keeping track
print("\n=== before cleaning summary ===")
print(f"Total rows (including non-images): {len(merged_df)}")
print("Images by split:")
display(merged_df.groupby("split").size().rename("images").reset_index())

print("Images by split & label:")
display(merged_df.groupby(["split","label"]).size().rename("images").reset_index().sort_values(["split","images"], ascending=[True,False]))

classes = sorted(merged_df["label"].dropna().unique().tolist())
print(f"Detected classes: {classes}")

# keep for next cells
inv_df = merged_df.copy()

Found 3 _classes.csv files.
Parsed label rows from CSVs: 520


filename    label
0  D19_2668fdbb-60a5-11ec-8402-0a7404972c70_jpg.r...  healthy
1  D19_2a83e1c9-60a5-11ec-8402-0a7404972c70_jpg.r...  healthy
2  D18_26750bd9-60a5-11ec-8402-0a7404972c70_jpg.r...  healthy
3  D0_0afdbb0b-60a5-11ec-8402-0a7404972c70_png.rf...  healthy
4  D19_2ab64e33-60a5-11ec-8402-0a7404972c70_jpg.r...  healthy


=== before cleaning summary ===
Total rows (including non-images): 523
Images by split:


split  images
0   test      21
1  train     481
2  valid      21

Images by split & label:


split label  images
0   test     d      20
1  train     d     480
2  valid     d      20

Detected classes: ['d']


In [6]:
# Fix placeholder 'd' labels with CSV labels
inv_df_fixed = inv_df.copy()

# Replace any placeholder 'd' with healthy if CSV label exists
inv_df_fixed['label'] = inv_df_fixed['label'].combine_first(inv_df_fixed['label'])

# If there are still 'd' labels, override them manually to 'healthy'
inv_df_fixed['label'] = inv_df_fixed['label'].replace({'d': 'healthy'})

# Update has_label column
inv_df_fixed['has_label'] = inv_df_fixed['label'].notna()

# Print updated distribution
print("=== Corrected before cleaning summary ===")
display(inv_df_fixed.groupby(['split', 'label']).size().rename('images').reset_index())
print("Detected classes:", sorted(inv_df_fixed['label'].dropna().unique().tolist()))

# Save corrected version
inv_df_fixed.to_csv(LOG_ROOT / "dataset1_inventory_corrected_labels.csv", index=False)
inv_df = inv_df_fixed.copy()

=== Corrected before cleaning summary ===


split    label  images
0   test  healthy      20
1  train  healthy     480
2  valid  healthy      20

Detected classes: ['healthy']


In [7]:
df = inv_df.copy()

# keeping only valid image extensions
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".tif", ".webp"}
df["ext"] = df["filename"].map(lambda s: Path(s).suffix.lower())
df = df[df["ext"].isin(IMG_EXTS) | df["ext"].eq("")]

# dropping unlabeled rows
df = df[df["has_label"]]

# validating images: dropping corrupt/unreadable images
def is_valid_image(path: str) -> bool:
    try:
        with Image.open(path) as im:
            im.verify()
        return True
    except Exception:
        return False

df["is_valid"] = [is_valid_image(p) for p in tqdm(df["src_path"], desc="Validating images")]
bad_df = df[~df["is_valid"]]
df = df[df["is_valid"]].drop(columns=["is_valid"])

# computing exact content hash
def sha256_of_file(path: str) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(1<<20), b""):
            h.update(chunk)
    return h.hexdigest()

df["sha256"] = [sha256_of_file(p) for p in tqdm(df["src_path"], desc="Hashing (SHA-256)")]

# marking duplicates
dup_mask = df.duplicated(subset=["sha256"], keep="first")
exact_dups_df = df[dup_mask]
df_nodup = df[~dup_mask]

bad_df.to_csv(LOG_ROOT / "removed_corrupt.csv", index=False)
exact_dups_df.to_csv(LOG_ROOT / "removed_exact_duplicates.csv", index=False)
df_nodup.to_csv(LOG_ROOT / "kept_after_exact_dups.csv", index=False)

# printing numbers for report
print("=== After exact-duplicate filtering ===")
print(f"Removed corrupt: {len(bad_df)}, Removed exact duplicates: {len(exact_dups_df)}")
display(df_nodup.groupby(['split','label']).size().rename('images').reset_index())

clean_step2_df = df_nodup.copy()

Hashing (SHA-256): 100%|██████████| 520/520 [00:00<00:00, 1496.02it/s]

=== After exact-duplicate filtering ===
Removed corrupt: 0, Removed exact duplicates: 0


split    label  images
0   test  healthy      20
1  train  healthy     480
2  valid  healthy      20

In [8]:
def compute_phash(path: str):
    try:
        with Image.open(path) as im:
            return imagehash.phash(im)
    except Exception:
        return None

tqdm.pandas(desc="Computing pHash")
tmp = clean_step2_df.copy()
tmp["phash"] = tmp["src_path"].progress_map(compute_phash)

# dropping rows where phash couldn't be computed
phash_df = tmp[tmp["phash"].notna()].copy()

# Marking near-duplicates
THRESH = 4  
keep_mask = np.ones(len(phash_df), dtype=bool)
hashes = phash_df["phash"].tolist()

for i in range(len(hashes)):
    if not keep_mask[i]:
        continue
    hi = hashes[i]
    for j in range(i+1, len(hashes)):
        if not keep_mask[j]:
            continue
        if (hi - hashes[j]) <= THRESH:
            keep_mask[j] = False

near_dups_df = phash_df[~keep_mask]
after_phash_df = phash_df[keep_mask]

near_dups_df.to_csv(LOG_ROOT / "removed_near_duplicates_phash.csv", index=False)
after_phash_df.to_csv(LOG_ROOT / "kept_after_phash.csv", index=False)

print(f"Near-duplicates removed (<= {THRESH} bits): {len(near_dups_df)}")
print(f"Remaining after pHash filter: {len(after_phash_df)}")
display(after_phash_df.groupby(['split','label']).size().rename('images').reset_index())

clean_step3_df = after_phash_df.copy()

Computing pHash: 100%|██████████| 520/520 [00:02<00:00, 197.85it/s]


Near-duplicates removed (<= 4 bits): 8
Remaining after pHash filter: 512


split    label  images
0   test  healthy      20
1  train  healthy     472
2  valid  healthy      20

In [9]:
def safe_label(label: str) -> str:
    return label.strip().lower().replace(" ", "_").replace("-", "_")

def unique_name(label: str, sha256: str, ext: str) -> str:
    return f"{label}_{sha256[:10]}{ext.lower() or '.jpg'}"

final_df = clean_step3_df.copy()
final_df["label"] = final_df["label"].map(safe_label)

# preparing output folders
for sp in final_df["split"].unique():
    for lb in final_df["label"].dropna().unique():
        (OUT_ROOT / sp / lb).mkdir(parents=True, exist_ok=True)

rows = []
for r in tqdm(final_df.itertuples(index=False), total=len(final_df), desc="Copying cleaned files"):
    src = Path(r.src_path)
    sp = r.split
    lb = r.label
    ext = Path(r.filename).suffix.lower()
    new_fname = unique_name(lb, r.sha256, ext)
    dst = OUT_ROOT / sp / lb / new_fname
    shutil.copy2(src, dst)
    rows.append({
        "split": sp,
        "label": lb,
        "src_path": str(src),
        "dst_path": str(dst),
        "sha256": r.sha256
    })

manifest = pd.DataFrame(rows)
manifest.to_csv(LOG_ROOT / "manifest_clean_dataset1.csv", index=False)

print(f"Clean dataset saved to: {OUT_ROOT}")

# final summary
display(final_df.groupby(['split','label']).size().rename('images').reset_index())
print("Classes:", sorted(final_df['label'].dropna().unique().tolist()))

Copying cleaned files: 100%|██████████| 512/512 [00:00<00:00, 1281.92it/s]


Clean dataset saved to: /kaggle/working/clean_dataset1


split    label  images
0   test  healthy      20
1  train  healthy     472
2  valid  healthy      20

Classes: ['healthy']


In [10]:
# Rename the cleaned dataset folder and download zip file
old_path = Path("/kaggle/working/clean_dataset1")
new_path = Path("/kaggle/working/healthyeyes512")

if old_path.exists():
    old_path.rename(new_path)
    print(f"Renamed folder to: {new_path}")
else:
    print("Folder already renamed or missing:", new_path)

zip_path = shutil.make_archive("/kaggle/working/healthyeyes512", 'zip', new_path)
print(f"Zipped dataset available at: {zip_path}")

Renamed folder to: /kaggle/working/healthyeyes512
Zipped dataset available at: /kaggle/working/healthyeyes512.zip


## Cleaning: Dataset 2 & 19 (they have the same structure)

In [11]:
try:
    from PIL import Image
except ImportError:
    !pip -q install pillow
    from PIL import Image

try:
    import imagehash 
except ImportError:
    pass

DATASET_NAME = "dataset19"              
OUT_NAME     = "clean_dataset19"           

ROOT_IN   = Path("/kaggle/input") / DATASET_NAME
assert ROOT_IN.exists(), f"Dataset not found: {ROOT_IN}"

SPLIT_ALIASES = {"val":"valid", "validation":"valid"}
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"}

# label normalization
LABEL_ALIASES = {
    "healthy": "healthy", "normal": "healthy",
    "ill": "ill", "sick": "ill",
    "bacterial": "bacterial_dermatosis", "bacterial_dermatosis": "bacterial_dermatosis",
    "fungal": "fungal_infection", "fungal_infection": "fungal_infection",
    "otitis": "ear_mites", "ear_mites": "ear_mites",
    "dental": "dental_disease", "dental_disease": "dental_disease",
    "paw_disease": "paw_injury", "paw_injury": "paw_injury",
}
FILENAME_LABEL_REGEX = re.compile(r"^[A-Za-z]+")

def norm_label(s: str|None) -> str|None:
    if not s: return None
    k = s.strip().lower().replace(" ", "_").replace("-", "_")
    return LABEL_ALIASES.get(k, k)

def find_splits(root: Path):
    splits = {}
    for p in root.iterdir():
        if not p.is_dir(): continue
        name = p.name.lower()
        name = SPLIT_ALIASES.get(name, name)
        if name in {"train","valid","test"}:
            splits[name] = p
    return splits

def parse_classes_csv(root: Path) -> pd.DataFrame:
    csvs = list(root.rglob("_classes.csv"))
    rows = []
    for csv in csvs:
        try:
            df = pd.read_csv(csv)
            df.columns = [c.strip().lower() for c in df.columns]
            if "filename" not in df.columns:
                print(f"Missing 'filename' in {csv}")
                continue
            class_cols = [c for c in df.columns if c != "filename"]
            if not class_cols:
                print(f"No class columns in {csv}")
                continue
            melted = df.melt(id_vars="filename", value_vars=class_cols,
                             var_name="label", value_name="val")
            pos = melted[melted["val"].astype(str).isin(["1","true","True"]) | (melted["val"]==1)].copy()
            if pos.empty and len(class_cols)==1:
                pos = df[["filename"]].copy()
                pos["label"] = class_cols[0]
            rows.append(pos[["filename","label"]])
        except Exception as e:
            print(f"Errorreading {csv}: {e}")
    if not rows:
        return pd.DataFrame(columns=["filename","label"])
    df = pd.concat(rows, ignore_index=True).drop_duplicates()
    df["filename_lower"] = df["filename"].str.lower()
    df["label"] = df["label"].map(norm_label)
    return df[["filename_lower","label"]]

def label_from_filename(name: str) -> str|None:
    m = FILENAME_LABEL_REGEX.match(Path(name).stem)
    return norm_label(m.group(0)) if m else None

def build_inventory(root: Path) -> pd.DataFrame:
    splits = find_splits(root)
    rows = []
    for split, split_dir in splits.items():
        for p in split_dir.rglob("*"):
            if not p.is_file(): continue
            rel = p.relative_to(split_dir)
            ext = p.suffix.lower()
            if ext and (ext not in IMG_EXTS):   
                continue
            parts = rel.parts
            label_dir = norm_label(parts[0]) if len(parts) > 1 else None  
            rows.append({
                "split": split,
                "src_path": str(p),
                "rel_from_split": str(rel),
                "label_dir": label_dir,
                "label_file": label_from_filename(p.name),
                "filename": p.name
            })
    df = pd.DataFrame(rows)
    if df.empty:
        print("Warning: No image-like files found.")
        return df
    df["filename_lower"] = df["filename"].str.lower()
    csv_df = parse_classes_csv(root)
    if not csv_df.empty:
        df = df.merge(csv_df, on="filename_lower", how="left", suffixes=("","_csv"))
        df["label"] = df["label"].combine_first(df["label_dir"]).combine_first(df["label_file"])
        label_source = np.where(df["label"].notna() & df["label"].eq(df.get("label", np.nan)), "csv",
                         np.where(df["label"].notna() & df["label"].eq(df["label_dir"]), "dir",
                         np.where(df["label"].notna() & df["label"].eq(df["label_file"]), "file", None)))
        df["label_source"] = label_source
    else:
        df["label"] = df["label_dir"].combine_first(df["label_file"])
        df["label_source"] = np.where(df["label"].eq(df["label_dir"]), "dir",
                               np.where(df["label"].eq(df["label_file"]), "file", None))
    df["has_label"] = df["label"].notna()
    return df

inv = build_inventory(ROOT_IN)

print(f"\nDataset: {DATASET_NAME}")
print(f"Total rows scanned: {len(inv):,}")
print("\n— Images by split —")
display(inv.groupby("split").size().rename("images").reset_index())

print("\n— Label source coverage —")
display(inv["label_source"].value_counts(dropna=False).rename("rows"))

print("\n— Unlabeled rows (will be dropped):", int((~inv["has_label"]).sum()))
if (~inv["has_label"]).any():
    display(inv.loc[~inv["has_label"], ["split","rel_from_split","filename"]].head(10))

print("\n— Class distribution (raw, before cleaning) —")
display(inv.groupby(["split","label"]).size().rename("images").reset_index().sort_values(["split","images"], ascending=[True,False]))

# the directory tree
def print_tree(root: Path, max_depth=2):
    for p, dirs, files in os.walk(root):
        rel = Path(p).relative_to(root)
        depth = len(rel.parts)
        if depth > max_depth:
            dirs[:] = []
            continue
        img_count = sum(f.lower().endswith(tuple(IMG_EXTS)) for f in files)
        indent = "  " * depth
        print(f"{indent}{rel.as_posix() or '.'}/  ({img_count} images)")
print("\n— Directory snapshot —")
print_tree(ROOT_IN, max_depth=2)

inv_df = inv.copy()
OUT_ROOT = Path("/kaggle/working") / OUT_NAME
LOG_ROOT = Path("/kaggle/working") / f"logs_{OUT_NAME}"
OUT_ROOT.mkdir(parents=True, exist_ok=True)
LOG_ROOT.mkdir(parents=True, exist_ok=True)


Dataset: dataset19
Total rows scanned: 149

— Images by split —


split  images
0   test      13
1  train     123
2  valid      13


— Label source coverage —


label_source
csv    149
Name: rows, dtype: int64


— Unlabeled rows (will be dropped): 0

— Class distribution (raw, before cleaning) —


split    label  images
0   test    fleas       9
1   test  healthy       4
2  train    fleas      66
3  train  healthy      57
5  valid  healthy       7
4  valid    fleas       6


— Directory snapshot —
./  (0 images)
  valid/  (13 images)
  test/  (13 images)
  train/  (123 images)


In [12]:
# filter to images + labeled
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"}
df = inv_df.copy()
df["ext"] = df["filename"].str.lower().map(lambda s: Path(s).suffix.lower())
df = df[df["ext"].isin(IMG_EXTS) | df["ext"].eq("")]
df = df[df["has_label"]].copy()

# validating images
def is_valid_image(path: str) -> bool:
    try:
        with Image.open(path) as im:
            im.verify()
        return True
    except Exception:
        return False

df["is_valid"] = [is_valid_image(p) for p in tqdm(df["src_path"], desc="Validating images")]
bad_df = df[~df["is_valid"]].copy()
df = df[df["is_valid"]].drop(columns=["is_valid"])

# exacting duplicates (SHA-256)
def sha256_of_file(path: str) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(1<<20), b""):
            h.update(chunk)
    return h.hexdigest()

df["sha256"] = [sha256_of_file(p) for p in tqdm(df["src_path"], desc="Hashing (SHA-256)")]
dup_mask = df.duplicated(subset=["sha256"], keep="first")
exact_dups = df[dup_mask].copy()
keep_df = df[~dup_mask].copy()

# Near-duplicates (pHash)
def compute_phash(path: str):
    try:
        with Image.open(path) as im:
            return imagehash.phash(im)
    except Exception:
        return None

tqdm.pandas(desc="Computing pHash")
keep_df["phash"] = keep_df["src_path"].progress_map(compute_phash)
phash_df = keep_df[keep_df["phash"].notna()].copy()

PHASH_THRESH = 4 
keep_mask = np.ones(len(phash_df), dtype=bool)
hashes = phash_df["phash"].tolist()
for i in range(len(hashes)):
    if not keep_mask[i]:
        continue
    hi = hashes[i]
    for j in range(i+1, len(hashes)):
        if not keep_mask[j]:
            continue
        if (hi - hashes[j]) <= PHASH_THRESH:
            keep_mask[j] = False

near_dups = phash_df[~keep_mask].copy()
final_df = phash_df[keep_mask].copy()

# export standardized structure
def safe_label(s: str) -> str:
    return s.strip().lower().replace(" ", "_").replace("-", "_")

final_df["label"] = final_df["label"].map(safe_label)

for sp in final_df["split"].unique():
    for lb in final_df["label"].dropna().unique():
        (OUT_ROOT / sp / lb).mkdir(parents=True, exist_ok=True)

rows = []
for r in tqdm(final_df.itertuples(index=False), total=len(final_df), desc="Copying cleaned files"):
    src = Path(r.src_path)
    sp  = r.split
    lb  = r.label
    ext = Path(r.filename).suffix.lower()
    new_name = f"{lb}_{r.sha256[:10]}{ext or '.jpg'}"
    dst = OUT_ROOT / sp / lb / new_name
    shutil.copy2(src, dst)
    rows.append({"split":sp, "label":lb, "src_path":str(src), "dst_path":str(dst), "sha256":r.sha256})

manifest = pd.DataFrame(rows)
manifest.to_csv(LOG_ROOT / "manifest.csv", index=False)
bad_df.to_csv(LOG_ROOT / "removed_corrupt.csv", index=False)
exact_dups.to_csv(LOG_ROOT / "removed_exact_duplicates.csv", index=False)
near_dups.to_csv(LOG_ROOT / "removed_near_duplicates_phash.csv", index=False)

print("\n=== Summary ===")
print(f"Original scanned rows: {len(inv_df)}")
print(f"Corrupt removed: {len(bad_df)}")
print(f"Exact duplicates removed: {len(exact_dups)}")
print(f"Near-duplicates removed (<= {PHASH_THRESH} bits): {len(near_dups)}")
print(f"Final kept: {len(final_df)}")
print("\n— Final by split & label —")
display(final_df.groupby(['split','label']).size().rename('images').reset_index().sort_values(['split','images'], ascending=[True,False]))

print("\nClean dataset saved to:", OUT_ROOT)
print("Logs & manifest at:", LOG_ROOT)

Copying cleaned files: 100%|██████████| 145/145 [00:00<00:00, 1362.64it/s]


=== Summary ===
Original scanned rows: 149
Corrupt removed: 0
Exact duplicates removed: 0
Near-duplicates removed (<= 4 bits): 4
Final kept: 145

— Final by split & label —


split    label  images
0   test    fleas       9
1   test  healthy       4
2  train    fleas      63
3  train  healthy      56
5  valid  healthy       7
4  valid    fleas       6


Clean dataset saved to: /kaggle/working/clean_dataset19
Logs & manifest at: /kaggle/working/logs_clean_dataset19


In [13]:
LOGS = Path("/kaggle/working/logs_clean_dataset19")
OUT  = Path("/kaggle/working/clean_dataset19")

manifest = pd.read_csv(LOGS/"manifest.csv")
final_n = len(manifest)

target_dir = OUT.parent / f"dataset19_clean_{final_n}"
if OUT.exists() and OUT != target_dir:
    OUT.rename(target_dir)
    print("Renamed to:", target_dir)
else:
    print("Using:", target_dir)

zip_path = shutil.make_archive(str(target_dir), 'zip', target_dir)
print("Zipped to:", zip_path)

Renamed to: /kaggle/working/dataset19_clean_145
Zipped to: /kaggle/working/dataset19_clean_145.zip


## CSV File Labels for Datasets 1,2,3,4 & 5

In [14]:
# finding manifests produced by our cleaning pipeline
manifest_paths = set(glob.glob("/kaggle/working/logs_*/manifest.csv"))
ds1_manifest = "/kaggle/working/logs_dataset1/manifest_clean_dataset1.csv"
if Path(ds1_manifest).exists():
    manifest_paths.add(ds1_manifest)

assert manifest_paths, "No manifests found in /kaggle/working/."

# making labels root
LABELS_ROOT = Path("/kaggle/working/labels")
LABELS_ROOT.mkdir(parents=True, exist_ok=True)

# normalize some label spellings
ALIASES = {
    "healthy":"healthy",
    "eye_redness":"eye_redness",
    "skin_lesion":"skin_lesion",
    "skin_lesions":"skin_lesion",
    "nasal_discharge":"nasal_discharge",
    "keratosis":"keratosis",
}

per_dataset_rows = []  

for mp in sorted(manifest_paths):
    p = Path(mp)
    df = pd.read_csv(p)
    ds_name = p.parent.name.replace("logs_", "")
    if ds_name == "dataset1":
        ds_name = "healthyeyes512"  

    df["label_norm"] = df["label"].str.strip().str.lower().map(lambda x: ALIASES.get(x, x))
    df["split"] = df["split"].str.lower()
    df["super_label"] = np.where(df["label_norm"] == "healthy", "healthy", "ill")

    out_dir = LABELS_ROOT / ds_name
    out_dir.mkdir(parents=True, exist_ok=True)

    stageA = df[["dst_path", "split", "label_norm", "super_label"]].copy()
    stageA["dataset_name"] = ds_name
    stageA["target"] = (stageA["super_label"] == "ill").astype(int)
    stageA_path = out_dir / f"stageA_labels_{ds_name}.csv"
    stageA.to_csv(stageA_path, index=False)

    stageB = df[df["super_label"] == "ill"][["dst_path", "split", "label_norm"]].copy()
    stageB["dataset_name"] = ds_name
    stageB = stageB.rename(columns={"label_norm": "disease"})
    stageB_path = out_dir / f"stageB_labels_{ds_name}.csv"
    stageB.to_csv(stageB_path, index=False)

    a_counts = stageA.groupby(["split","super_label"]).size().rename("images").reset_index()
    b_counts = stageB.groupby(["split","disease"]).size().rename("images").reset_index()
    per_dataset_rows.append({
        "dataset_name": ds_name,
        "stageA_path": str(stageA_path),
        "stageB_path": str(stageB_path),
        "stageA_total": len(stageA),
        "stageB_total": len(stageB),
        "stageA_by_split": a_counts.to_dict(orient="records"),
        "stageB_by_split": b_counts.to_dict(orient="records"),
    })

index_rows = []
for r in per_dataset_rows:
    index_rows.append({
        "dataset_name": r["dataset_name"],
        "stageA_path": r["stageA_path"],
        "stageB_path": r["stageB_path"],
        "stageA_total": r["stageA_total"],
        "stageB_total": r["stageB_total"],
    })
index_df = pd.DataFrame(index_rows)
index_csv = LABELS_ROOT / "labels_index.csv"
index_df.to_csv(index_csv, index=False)

summary_json = LABELS_ROOT / "labels_summary.json"
with open(summary_json, "w") as f:
    json.dump(per_dataset_rows, f, indent=2)

print("Wrote per-dataset labels to:", LABELS_ROOT)
print("Index:", index_csv)
print("Summary:", summary_json)

zip_path = shutil.make_archive("/kaggle/working/labels_all_datasets", "zip", "/kaggle/working/labels")
print("Labels ZIP at:", zip_path)

Wrote per-dataset labels to: /kaggle/working/labels
Index: /kaggle/working/labels/labels_index.csv
Summary: /kaggle/working/labels/labels_summary.json
Labels ZIP at: /kaggle/working/labels_all_datasets.zip


## Cleaning: Dataset 7

In [15]:
DATASET_NAME = "dataset7"  
ROOT_IN = Path("/kaggle/input")/DATASET_NAME
assert ROOT_IN.exists(), f"Not found: {ROOT_IN}"

SPLIT_ALIASES = {"val":"valid","validation":"valid"}
IMG_EXTS = {".jpg",".jpeg",".png",".bmp",".tif",".tiff",".webp"}

# normalize class names
ALIASES = {
    "healthy":"healthy", "normal":"healthy",
    "dermatitis":"dermatitis",
    "fungal_infections":"fungal_infection", "fungal_infection":"fungal_infection",
    "hypersensitivity":"hypersensitivity",
    "ringworm":"ringworm",
    "demodicosis":"demodicosis",
}
def norm_label(s):
    if s is None: return None
    k = str(s).strip().lower().replace(" ","_")
    return ALIASES.get(k, k)

def find_splits(root):
    out={}
    for p in root.iterdir():
        if p.is_dir():
            name = SPLIT_ALIASES.get(p.name.lower(), p.name.lower())
            if name in {"train","valid","test"}:
                out[name]=p
    if not out: out["train"]=root
    return out

rows=[]
for split, sdir in find_splits(ROOT_IN).items():
    for cls in sdir.iterdir():
        if not cls.is_dir(): continue
        label = norm_label(cls.name)
        for img in cls.rglob("*"):
            if img.is_file() and img.suffix.lower() in IMG_EXTS:
                rows.append({"split":split, "label":label, "src_path":str(img)})
df = pd.DataFrame(rows)
print(f"Dataset: {DATASET_NAME}  |  images found: {len(df)}")
print("\n— Images by split —")
display(df.groupby("split").size().rename("images").reset_index())
print("\n— Class distribution —")
display(df.groupby(["split","label"]).size().rename("images").reset_index().sort_values(["split","images"], ascending=[True,False]))

def thumbs(_df, per=3, max_classes=6):
    if _df.empty: return
    sample = (_df.groupby("label", group_keys=False)
                .apply(lambda x: x.sample(min(per, len(x)), random_state=42))
                .reset_index(drop=True)
                .sort_values("label")
                .head(per*max_classes))
    n=len(sample); cols=per; rows=int(np.ceil(n/cols))
    plt.figure(figsize=(3.4*cols, 3.4*rows))
    for i,(p,lb) in enumerate(zip(sample["src_path"], sample["label"]), start=1):
        try:
            im = Image.open(p); plt.subplot(rows,cols,i); plt.imshow(im); plt.title(lb); plt.axis("off")
        except: pass
    plt.tight_layout(); plt.show()

INV_DF = df.copy()

Dataset: dataset7  |  images found: 4315

— Images by split —


split  images
0   test     433
1  train    3022
2  valid     860


— Class distribution —


split             label  images
5    test          ringworm     115
0    test       demodicosis     100
3    test           healthy      69
1    test        dermatitis      66
2    test  fungal_infection      54
4    test  hypersensitivity      29
11  train          ringworm     791
6   train       demodicosis     588
7   train        dermatitis     546
9   train           healthy     492
8   train  fungal_infection     375
10  train  hypersensitivity     230
17  valid          ringworm     212
13  valid        dermatitis     175
12  valid       demodicosis     174
15  valid           healthy     139
14  valid  fungal_infection      97
16  valid  hypersensitivity      63

In [16]:
labels = sorted(manifest["label"].unique())

for sp, g in manifest.groupby("split"):
    g = g.copy()
    g["filename"] = g["dst_path"].apply(lambda s: Path(s).name)
    wide = (pd.get_dummies(g["label"])
              .reindex(columns=labels, fill_value=0)
              .astype(int))
    wide.insert(0, "filename", g["filename"].values)
    (OUT_ROOT/sp).mkdir(parents=True, exist_ok=True)
    wide.to_csv(OUT_ROOT/sp/"_classes.csv", index=False)

print("Wrote _classes.csv to each split in:", OUT_ROOT)

ds_name = f"clean_{DATASET_NAME}"
LABELS_ROOT = Path("/kaggle/working/labels"); LABELS_ROOT.mkdir(parents=True, exist_ok=True)
out_dir = LABELS_ROOT/ds_name; out_dir.mkdir(parents=True, exist_ok=True)

m = manifest.copy()
m["label_norm"]  = m["label"].str.strip().str.lower()
m["super_label"] = (m["label_norm"] != "healthy").map({True:"ill", False:"healthy"})

# Stage A: healthy vs ill
stageA = m[["dst_path","split","label_norm","super_label"]].copy()
stageA["dataset_name"] = ds_name
stageA["target"] = (stageA["super_label"]=="ill").astype(int)
a_path = out_dir/f"stageA_labels_{ds_name}.csv"; stageA.to_csv(a_path, index=False)

# Stage B: ill-only by disease
stageB = m[m["super_label"]=="ill"][["dst_path","split","label_norm"]].copy()
stageB["dataset_name"] = ds_name
stageB.rename(columns={"label_norm":"disease"}, inplace=True)
b_path = out_dir/f"stageB_labels_{ds_name}.csv"; stageB.to_csv(b_path, index=False)

# updating the labels index
idx_path = LABELS_ROOT/"labels_index.csv"
try:
    idx_df = pd.read_csv(idx_path)
    idx_df = idx_df[idx_df["dataset_name"] != ds_name]
except FileNotFoundError:
    idx_df = pd.DataFrame(columns=["dataset_name","stageA_path","stageB_path","stageA_total","stageB_total"])
idx_df = pd.concat([idx_df, pd.DataFrame([{
    "dataset_name": ds_name,
    "stageA_path": str(a_path),
    "stageB_path": str(b_path),
    "stageA_total": len(stageA),
    "stageB_total": len(stageB),
}])], ignore_index=True)
idx_df.to_csv(idx_path, index=False)
print("Updated labels index:", idx_path)

Wrote _classes.csv to each split in: /kaggle/working/clean_dataset19
Updated labels index: /kaggle/working/labels/labels_index.csv


In [17]:
try:
    INV_DF 
except NameError:
    IMG_EXTS = {".jpg",".jpeg",".png",".bmp",".tif",".tiff",".webp"}
    SPLIT_ALIASES = {"val":"valid","validation":"valid"}
    def find_splits(root):
        out={}
        for p in root.iterdir():
            if p.is_dir():
                name = SPLIT_ALIASES.get(p.name.lower(), p.name.lower())
                if name in {"train","valid","test"}: out[name]=p
        if not out: out["train"]=root
        return out
    rows=[]
    for split, sdir in find_splits(ROOT_IN).items():
        for cls in sdir.iterdir():
            if not cls.is_dir(): continue
            label = cls.name.strip().lower().replace(" ","_")
            for img in cls.rglob("*"):
                if img.is_file() and img.suffix.lower() in IMG_EXTS:
                    rows.append({"split":split,"label":label,"src_path":str(img)})
    INV_DF = pd.DataFrame(rows)

OUT_ROOT = Path(f"/kaggle/working/clean_{DATASET_NAME}")
LOG_ROOT = Path(f"/kaggle/working/logs_clean_{DATASET_NAME}")
OUT_ROOT.mkdir(parents=True, exist_ok=True)
LOG_ROOT.mkdir(parents=True, exist_ok=True)

df = INV_DF.copy()

# corruption check
def is_valid(p):
    try:
        with Image.open(p) as im: im.verify()
        return True
    except Exception:
        return False

df["is_valid"] = [is_valid(p) for p in tqdm(df["src_path"], desc="Validating")]
bad = df[~df["is_valid"]].copy()
df = df[df["is_valid"]].drop(columns=["is_valid"])

# exacting duplicates (SHA-256)
def sha256(p):
    h=hashlib.sha256()
    with open(p,"rb") as f:
        for chunk in iter(lambda: f.read(1<<20), b""): h.update(chunk)
    return h.hexdigest()

df["sha256"] = [sha256(p) for p in tqdm(df["src_path"], desc="SHA-256")]
dup_mask = df.duplicated("sha256", keep="first")
exact = df[dup_mask].copy()
keep  = df[~dup_mask].copy()

# near-duplicates (pHash)
try:
    import imagehash
except ImportError:
    !pip -q install imagehash
    import imagehash

def phash(p):
    try:
        with Image.open(p) as im: return imagehash.phash(im)
    except Exception:
        return None

tqdm.pandas(desc="pHash")
keep["phash"] = keep["src_path"].progress_map(phash)
keep = keep[keep["phash"].notna()].copy()

PHASH_THRESH = 4
mask = np.ones(len(keep), dtype=bool)
H = keep["phash"].tolist()
for i in range(len(H)):
    if not mask[i]: continue
    for j in range(i+1, len(H)):
        if not mask[j]: continue
        if (H[i] - H[j]) <= PHASH_THRESH:
            mask[j] = False

near  = keep[~mask].copy()
final = keep[mask].copy()

# copying standardized layout + manifest
for sp in final["split"].unique():
    for lb in final["label"].unique():
        (OUT_ROOT/sp/lb).mkdir(parents=True, exist_ok=True)

rows=[]
for r in tqdm(final.itertuples(index=False), total=len(final), desc="Copying cleaned files"):
    src = Path(r.src_path); ext = src.suffix.lower() or ".jpg"
    new_name = f"{r.label}_{r.sha256[:10]}{ext}"
    dst = OUT_ROOT / r.split / r.label / new_name
    shutil.copy2(src, dst)
    rows.append({"split":r.split,"label":r.label,"src_path":str(src),"dst_path":str(dst),"sha256":r.sha256})

manifest = pd.DataFrame(rows)
manifest.to_csv(LOG_ROOT/"manifest.csv", index=False)
bad.to_csv(LOG_ROOT/"removed_corrupt.csv", index=False)
exact.to_csv(LOG_ROOT/"removed_exact_duplicates.csv", index=False)
near.to_csv(LOG_ROOT/"removed_near_duplicates_phash.csv", index=False)

# writing Roboflow-style _classes.csv from manifest
labels = sorted(manifest["label"].unique())
for sp, g in manifest.groupby("split"):
    g = g.copy()
    g["filename"] = g["dst_path"].apply(lambda s: Path(s).name)
    wide = (pd.get_dummies(g["label"])
              .reindex(columns=labels, fill_value=0)
              .astype(int))
    wide.insert(0, "filename", g["filename"].values)
    (OUT_ROOT/sp).mkdir(parents=True, exist_ok=True)
    wide.to_csv(OUT_ROOT/sp/"_classes.csv", index=False)

# Stage-A / Stage-B CSVs for this dataset
ds_name = f"clean_{DATASET_NAME}"
LABELS_ROOT = Path("/kaggle/working/labels"); LABELS_ROOT.mkdir(parents=True, exist_ok=True)
out_dir = LABELS_ROOT/ds_name; out_dir.mkdir(parents=True, exist_ok=True)

m = manifest.copy()
m["label_norm"]  = m["label"].str.strip().str.lower()
m["super_label"] = np.where(m["label_norm"]=="healthy", "healthy", "ill")

stageA = m[["dst_path","split","label_norm","super_label"]].copy()
stageA["dataset_name"] = ds_name
stageA["target"] = (stageA["super_label"]=="ill").astype(int)
a_path = out_dir/f"stageA_labels_{ds_name}.csv"; stageA.to_csv(a_path, index=False)

stageB = m[m["super_label"]=="ill"][["dst_path","split","label_norm"]].copy()
stageB["dataset_name"] = ds_name
stageB.rename(columns={"label_norm":"disease"}, inplace=True)
b_path = out_dir/f"stageB_labels_{ds_name}.csv"; stageB.to_csv(b_path, index=False)

# updating labels index
idx_path = LABELS_ROOT/"labels_index.csv"
try:
    idx_df = pd.read_csv(idx_path)
    idx_df = idx_df[idx_df["dataset_name"] != ds_name]
except FileNotFoundError:
    idx_df = pd.DataFrame(columns=["dataset_name","stageA_path","stageB_path","stageA_total","stageB_total"])
idx_df = pd.concat([idx_df, pd.DataFrame([{
    "dataset_name": ds_name,
    "stageA_path": str(a_path),
    "stageB_path": str(b_path),
    "stageA_total": len(stageA),
    "stageB_total": len(stageB),
}])], ignore_index=True)
idx_df.to_csv(idx_path, index=False)

print("\n=== cleaning summary ===")
print(f"Original scanned: {len(INV_DF)}")
print(f"Corrupt removed: {len(bad)}")
print(f"Exact dups removed: {len(exact)}")
print(f"Near-dups removed (<= {PHASH_THRESH} bits): {len(near)}")
print(f"Final kept: {len(manifest)}")
print("\nFinal by split & label:")
display(manifest.groupby(['split','label']).size().rename('images').reset_index())

Copying cleaned files: 100%|██████████| 3781/3781 [00:05<00:00, 691.22it/s]



=== cleaning summary ===
Original scanned: 4315
Corrupt removed: 0
Exact dups removed: 82
Near-dups removed (<= 4 bits): 452
Final kept: 3781

Final by split & label:


split             label  images
0    test       demodicosis     100
1    test        dermatitis      60
2    test  fungal_infection      50
3    test           healthy      59
4    test  hypersensitivity      27
5    test          ringworm     113
6   train       demodicosis     584
7   train        dermatitis     484
8   train  fungal_infection     284
9   train           healthy     279
10  train  hypersensitivity     167
11  train          ringworm     738
12  valid       demodicosis     174
13  valid        dermatitis     173
14  valid  fungal_infection      91
15  valid           healthy     126
16  valid  hypersensitivity      61
17  valid          ringworm     211

In [18]:
n = len(pd.read_csv(f"/kaggle/working/logs_clean_{DATASET_NAME}/manifest.csv"))
target = OUT_ROOT.parent / f"{DATASET_NAME}_clean_{n}"
if OUT_ROOT.exists() and OUT_ROOT != target:
    OUT_ROOT.rename(target); print("Renamed to:", target)
zip_path = shutil.make_archive(str(target), "zip", target)
print("Zipped to:", zip_path)

Renamed to: /kaggle/working/dataset7_clean_3781
Zipped to: /kaggle/working/dataset7_clean_3781.zip


## Cleaning: Datasets 15, 17 & 20

In [19]:
DATASET_NAMES = ["dataset15", "dataset17", "dataset20"]
SPLITS        = ["train", "valid", "test"]
MIN_SIDE      = 128                
HAMMING_MAX   = 4                   
INPUT_ROOT    = Path("/kaggle/input")
WORK_ROOT     = Path("/kaggle/working")

def locate_dataset_root(ds_name: str) -> Path | None:
    cand = INPUT_ROOT / ds_name
    def is_ok(p: Path) -> bool:
        return (p / "train").exists() and (p / "valid").exists() and (p / "test").exists()
    if is_ok(cand):
        return cand
    for p in INPUT_ROOT.iterdir():
        if p.is_dir():
            q = p / ds_name
            if is_ok(q): return q
            for sub in p.iterdir():
                if sub.is_dir():
                    q = sub / ds_name
                    if is_ok(q): return q
    return None

def sha256_file(path: Path, chunk=1<<20) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for c in iter(lambda: f.read(chunk), b""):
            h.update(c)
    return h.hexdigest()

def image_ok(path: Path) -> tuple[bool,int,int]:
    try:
        with Image.open(path) as im:
            im.verify()
        with Image.open(path) as im:
            w, h = im.size
        return True, w, h
    except (UnidentifiedImageError, OSError, ValueError):
        return False, 0, 0

def phash_for(path: Path) -> imagehash.ImageHash:
    with Image.open(path) as im:
        return imagehash.phash(im.convert("RGB"))

def normalize_name(p: Path) -> str:
    stem = re.sub(r"\s+", "_", p.stem)
    stem = re.sub(r"[^A-Za-z0-9._-]", "", stem)
    return stem + p.suffix.lower()

def list_classes(split_dir: Path):
    return sorted([p.name for p in split_dir.iterdir() if p.is_dir()])

In [20]:
def clean_dataset(ds_name: str):
    base = locate_dataset_root(ds_name)
    if base is None:
        print(f"Skip: Could not find {ds_name} under {INPUT_ROOT} with train/valid/test.")
        return None

    print(f"\n=== Processing {ds_name} ===")
    for sp in SPLITS:
        print(sp, "classes:", list_classes(base / sp))

    clean_dir   = WORK_ROOT / f"{ds_name}_clean"
    removed_dir = WORK_ROOT / f"{ds_name}_removed"
    csv_dir     = WORK_ROOT / f"{ds_name}_clean_csvs"
    for d in [clean_dir, removed_dir, csv_dir]:
        if d.exists(): shutil.rmtree(d)
        d.mkdir(parents=True, exist_ok=True)

    # scanning & validating
    records, removed = [], []
    for sp in SPLITS:
        for cls in list_classes(base / sp):
            for img in tqdm(list((base / sp / cls).rglob("*")), desc=f"scan:{ds_name}:{sp}/{cls}"):
                if not img.is_file():
                    continue
                if img.suffix.lower() not in {".jpg",".jpeg",".png",".webp",".bmp",".tif",".tiff"}:
                    removed.append(dict(filepath=str(img), reason="non_image"))
                    continue
                ok, w, h = image_ok(img)
                if not ok:
                    removed.append(dict(filepath=str(img), reason="corrupt"))
                    continue
                if min(w, h) < MIN_SIDE:
                    removed.append(dict(filepath=str(img), reason=f"too_small<{MIN_SIDE}", width=w, height=h))
                    continue
                records.append(dict(split=sp, label=cls, filepath=str(img), width=w, height=h, sha256=sha256_file(img)))
    print(f"[{ds_name}] Kept (pre-dup): {len(records)} | Removed (pre-dup): {len(removed)}")

    # duplicate detection (pHash)
    seen, to_remove = [], set()
    for i, r in tqdm(list(enumerate(records)), desc=f"phash:{ds_name}"):
        p = Path(r["filepath"])
        try:
            h = phash_for(p)
            records[i]["phash"] = str(h)
        except Exception:
            removed.append(dict(filepath=str(p), reason="phash_error"))
            to_remove.add(i)
            continue
        dup = False
        for j, hprev in seen:
            if (h - hprev) <= HAMMING_MAX:
                to_remove.add(i)
                dup = True
                break
        if not dup:
            seen.append((i, h))

    for i in sorted(to_remove):
        removed.append(dict(filepath=records[i]["filepath"], reason=f"duplicate_phash<= {HAMMING_MAX}"))
    records = [r for k, r in enumerate(records) if k not in to_remove]
    print(f"[{ds_name}] After dup-filter — kept: {len(records)} | dropped dups: {len(to_remove)}")

    # materializing copies
    def copy_preserve(src: Path, base_dst: Path) -> Path:
        rel = src.relative_to(base)
        dst_dir = base_dst / rel.parent
        dst_dir.mkdir(parents=True, exist_ok=True)
        dst = dst_dir / normalize_name(Path(rel.name))
        if dst.exists():
            dst = dst_dir / (dst.stem + "_" + sha256_file(src)[:8] + dst.suffix)
        shutil.copy2(src, dst)
        return dst

    def copy_removed(src: Path, reason: str) -> Path:
        rel = src.relative_to(base)
        dst_dir = removed_dir / reason / rel.parent
        dst_dir.mkdir(parents=True, exist_ok=True)
        dst = dst_dir / normalize_name(Path(rel.name))
        if dst.exists():
            dst = dst_dir / (dst.stem + "_" + sha256_file(src)[:8] + dst.suffix)
        try:
            shutil.copy2(src, dst)
        except Exception:
            pass
        return dst

    for r in tqdm(records, desc=f"copy_keep:{ds_name}"):
        dst = copy_preserve(Path(r["filepath"]), clean_dir)
        r["clean_path"] = str(dst)
    for rm in tqdm(removed, desc=f"copy_removed:{ds_name}"):
        try:
            outp = copy_removed(Path(rm["filepath"]), rm["reason"])
            rm["copied_to"] = str(outp)
        except Exception:
            rm["copied_to"] = ""

    # CSVs & one-hot encoding
    df = pd.DataFrame(records)
    csv_dir.mkdir(parents=True, exist_ok=True)

    for sp in SPLITS:
        d = df[df["split"] == sp].copy()
        d = d[["clean_path","label","width","height","sha256","phash"]]
        out_csv = csv_dir / f"{ds_name}_{sp}.csv"
        d.to_csv(out_csv, index=False)

    classes = sorted(df["label"].unique())
    (csv_dir / f"{ds_name}_classes.txt").write_text("\n".join(classes), encoding="utf-8")

    rem_df = pd.DataFrame(removed)
    if not rem_df.empty:
        rem_df.to_csv(csv_dir / f"{ds_name}_removed_log.csv", index=False)

    onehot = df.copy()
    onehot["filename"] = onehot["clean_path"].apply(lambda p: Path(p).name)
    for c in classes:
        onehot[c] = (onehot["label"] == c).astype(int)
    onehot = onehot[["filename"] + classes]
    onehot.to_csv(csv_dir / f"{ds_name}_classes_onehot.csv", index=False)

    # rename with count & zip file
    final_count = len(df)
    clean_renamed = WORK_ROOT / f"{ds_name}_clean_{final_count}"
    csv_renamed   = WORK_ROOT / f"{ds_name}_clean_csvs_{final_count}"
    if clean_renamed.exists(): shutil.rmtree(clean_renamed)
    if csv_renamed.exists(): shutil.rmtree(csv_renamed)
    shutil.move(str(clean_dir), str(clean_renamed))
    shutil.move(str(csv_dir),  str(csv_renamed))

    zip_path = WORK_ROOT / f"{ds_name}_clean_{final_count}.zip"
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
        for root, _, files in os.walk(clean_renamed):
            for f in files:
                fp = Path(root) / f
                z.write(fp, arcname=str(fp.relative_to(WORK_ROOT)))
        for root, _, files in os.walk(csv_renamed):
            for f in files:
                fp = Path(root) / f
                z.write(fp, arcname=str(fp.relative_to(WORK_ROOT)))

    # summary
    split_counts = df.groupby("split").size().to_dict()
    print(f"[DONE {ds_name}] kept={final_count}  splits={split_counts}  zip={zip_path}")
    return {
        "name": ds_name,
        "kept": final_count,
        "split_counts": split_counts,
        "zip": str(zip_path),
        "clean_dir": str(clean_renamed),
        "csv_dir": str(csv_renamed),
    }

In [21]:
summaries = []
for name in DATASET_NAMES:
    try:
        s = clean_dataset(name)
        if s is not None:
            summaries.append(s)
    except Exception as e:
        print(f"[ERROR] {name}: {e}")

print("\n=== Summary ===")
for s in summaries:
    print(f"{s['name']}: kept={s['kept']} | splits={s['split_counts']} | zip={s['zip']}")


=== Processing dataset15 ===
train classes: ['flea_allergy', 'hotspot', 'mange', 'ringworm']
valid classes: ['flea_allergy', 'hotspot', 'mange', 'ringworm']
test classes: ['flea_allergy', 'hotspot', 'mange', 'ringworm']


scan:dataset15:test/ringworm: 100%|██████████| 13/13 [00:00<00:00, 182.28it/s]


[dataset15] Kept (pre-dup): 359 | Removed (pre-dup): 0


phash:dataset15: 100%|██████████| 359/359 [00:02<00:00, 152.17it/s]


[dataset15] After dup-filter — kept: 358 | dropped dups: 1


...

[dataset20] Kept (pre-dup): 2879 | Removed (pre-dup): 0


phash:dataset20: 100%|██████████| 2879/2879 [00:14<00:00, 198.26it/s]


[dataset20] After dup-filter — kept: 2628 | dropped dups: 251


copy_removed:dataset20: 100%|██████████| 251/251 [00:00<00:00, 830.47it/s]


[DONE dataset20] kept=2628  splits={'test': 116, 'train': 2037, 'valid': 475}  zip=/kaggle/working/dataset20_clean_2628.zip

=== Summary ===
dataset15: kept=358 | splits={'test': 36, 'train': 251, 'valid': 71} | zip=/kaggle/working/dataset15_clean_358.zip
dataset17: kept=3968 | splits={'test': 361, 'train': 2965, 'valid': 642} | zip=/kaggle/working/dataset17_clean_3968.zip
dataset20: kept=2628 | splits={'test': 116, 'train': 2037, 'valid': 475} | zip=/kaggle/working/dataset20_clean_2628.zip


## Cleaning: Dataset 21

In [22]:
BASE = Path("/kaggle/input/dataset21")
DS_ROOT = BASE / "dog-skin-5di"
IM_DIR = DS_ROOT / "images"
LB_DIR = DS_ROOT / "labels"
YAML = DS_ROOT / "data.yaml"
assert IM_DIR.exists() and LB_DIR.exists() and YAML.exists(), "Check dataset21 paths."

# load class names
with open(YAML, "r") as f:
    data_cfg = yaml.safe_load(f)
names = data_cfg.get("names", [])
assert names, "No class names found in data.yaml"

def norm(s): return re.sub(r"[\W_]+","_", s.strip().lower())
healthy_ids = {i for i,n in enumerate(names) if norm(n) in {"healthy","normal"}}
print("Classes:", names, "| healthy_ids:", healthy_ids)

# helpers
IMG_EXTS = {".jpg",".jpeg",".png",".webp",".bmp",".tif",".tiff",".jfif"}
KEEP_MIXED_AS_ILL_FOR_STAGE_A = True

def image_valid(p: Path):
    try:
        with Image.open(p) as im: im.verify()
        with Image.open(p) as im: w,h = im.size
        return True, w, h
    except (UnidentifiedImageError, OSError, ValueError):
        return False, 0, 0

# parser: only uses class ID, ignores extra columns
def read_label_ids(txt: Path):
    ids = []
    with open(txt, "r") as f:
        for ln in f:
            ln = ln.strip()
            if not ln: continue
            parts = ln.split()
            try:
                cid = int(parts[0])
            except ValueError:
                return None, "parse_error"
            ids.append(cid)
    return ids, None

# scanning & assign Stage A/B 
records, excluded = [], []
for img in tqdm(list(IM_DIR.rglob("*")), desc="scan_images"):
    if not (img.is_file() and img.suffix.lower() in IMG_EXTS):
        continue
    ok, w, h = image_valid(img)
    if not ok:
        excluded.append(dict(filename=img.name, reason="corrupt_image")); continue

    txt = LB_DIR / (img.stem + ".txt")
    if not txt.exists():
        excluded.append(dict(filename=img.name, reason="missing_label")); continue

    ids, err = read_label_ids(txt)
    if err is not None or ids is None or len(ids)==0:
        excluded.append(dict(filename=img.name, reason=err or "empty_label")); continue

    if not all(0 <= cid < len(names) for cid in ids):
        excluded.append(dict(filename=img.name, reason="class_id_out_of_range")); continue

    u = set(ids)
    if len(u) == 1:
        cid = next(iter(u))
        if cid in healthy_ids:
            stageA, stageB = "healthy", None
        else:
            stageA, stageB = "ill", names[cid]
    else:
        # keep as ill for Stage A if mixed
        if KEEP_MIXED_AS_ILL_FOR_STAGE_A:
            stageA, stageB = "ill", None
        else:
            excluded.append(dict(filename=img.name, reason="mixed_classes")); continue

    records.append(dict(
        filename=img.name,
        image_path=str(img),
        width=w, height=h,
        stageA=stageA,
        stageB=stageB or ""
    ))

# stage A/B DataFrames
df = pd.DataFrame(records)
assert not df.empty, "No usable images after validation."
dfA = df.copy(); dfA["label"] = dfA["stageA"]
dfB = df[(df["stageA"]=="ill") & (df["stageB"]!="")].copy(); dfB["label"] = dfB["stageB"]

# stratified split
def stratified_split(df_in, label_col="label", seed=42):
    idx = np.arange(len(df_in)); y = df_in[label_col].values
    sss1 = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)
    tr_idx, hold_idx = next(sss1.split(idx, y))
    y_hold = y[hold_idx]
    sss2 = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=seed)
    va_rel, te_rel = next(sss2.split(hold_idx, y_hold))
    va_idx, te_idx = hold_idx[va_rel], hold_idx[te_rel]
    split = pd.Series(index=df_in.index, dtype="object")
    split.iloc[tr_idx] = "train"; split.iloc[va_idx] = "valid"; split.iloc[te_idx] = "test"
    return split

dfA["split"] = stratified_split(dfA, "label")
if not dfB.empty and dfB["label"].nunique() > 1:
    dfB["split"] = stratified_split(dfB, "label")
else:
    dfB["split"] = "train"

OUT = Path("/kaggle/working/dataset21_classification")
if OUT.exists(): shutil.rmtree(OUT)
OUT.mkdir(parents=True, exist_ok=True)

def copy_to_split(df_in, stage_name):
    base_dir = OUT / stage_name
    for sp in ["train","valid","test"]:
        sub = df_in[df_in["split"]==sp]
        if sub.empty: continue
        for cls in sorted(sub["label"].unique()):
            (base_dir / sp / cls).mkdir(parents=True, exist_ok=True)
        for _, r in tqdm(sub.iterrows(), total=len(sub), desc=f"copy_{stage_name}:{sp}"):
            src = Path(r["image_path"])
            dst = base_dir / sp / r["label"] / src.name
            if dst.exists():
                dst = dst.with_name(dst.stem + "_" + hashlib.sha256(src.read_bytes()).hexdigest()[:8] + dst.suffix)
            shutil.copy2(src, dst)
    return base_dir

baseA = copy_to_split(dfA, "stageA_binary")
baseB = copy_to_split(dfB, "stageB_multiclass") if not dfB.empty else None

pd.DataFrame(excluded).to_csv(OUT / "excluded_log.csv", index=False)

Classes: ['bacterial-dermatosis', 'fungal-infection', 'healthy', 'hypersensitivity-allergic-dermatosis', 'demodicosis', 'ringworm'] | healthy_ids: {2}


copy_stageB_multiclass:test: 100%|██████████| 306/306 [00:00<00:00, 943.71it/s]


In [23]:
OUT = Path("/kaggle/working/dataset21_classification")
assert OUT.exists()

def write_csvs(base_dir: Path, prefix: str):
    recs = []
    for sp in ["train","valid","test"]:
        sp_dir = base_dir / sp
        if not sp_dir.exists(): continue
        for cls_dir in sp_dir.iterdir():
            if not cls_dir.is_dir(): continue
            for img in cls_dir.iterdir():
                if img.is_file():
                    recs.append(dict(split=sp, label=cls_dir.name, path=str(img), filename=img.name))
    df = pd.DataFrame(recs)
    if df.empty:
        print(f"No files under {base_dir}, skipping CSVs.")
        return None, None

    csv_dir = base_dir / "csvs"
    csv_dir.mkdir(parents=True, exist_ok=True)

    for sp in ["train","valid","test"]:
        d = df[df["split"]==sp][["path","label","filename"]]
        if not d.empty:
            p = csv_dir / f"{prefix}_{sp}.csv"
            d.to_csv(p, index=False)
            print("wrote", p)

    classes = sorted(df["label"].unique())
    (csv_dir / f"{prefix}_classes.txt").write_text("\n".join(classes), encoding="utf-8")

    oh = df.copy()
    for c in classes: oh[c] = (oh["label"]==c).astype(int)
    onehot = oh[["filename"] + classes].drop_duplicates("filename")
    onehot_p = csv_dir / f"{prefix}_classes_onehot.csv"
    onehot.to_csv(onehot_p, index=False)
    print("wrote", onehot_p)

    zip_path = base_dir.with_suffix(".zip")
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
        for root, _, files in os.walk(base_dir):
            for f in files:
                fp = Path(root) / f
                z.write(fp, arcname=str(fp.relative_to(base_dir.parent)))
    return df, zip_path

dfA, zipA = write_csvs(OUT / "stageA_binary", "dataset21_stageA")
dfB, zipB = write_csvs(OUT / "stageB_multiclass", "dataset21_stageB")

wrote /kaggle/working/dataset21_classification/stageA_binary/csvs/dataset21_stageA_train.csv
wrote /kaggle/working/dataset21_classification/stageA_binary/csvs/dataset21_stageA_valid.csv
wrote /kaggle/working/dataset21_classification/stageA_binary/csvs/dataset21_stageA_test.csv
wrote /kaggle/working/dataset21_classification/stageA_binary/csvs/dataset21_stageA_classes_onehot.csv
wrote /kaggle/working/dataset21_classification/stageB_multiclass/csvs/dataset21_stageB_train.csv
wrote /kaggle/working/dataset21_classification/stageB_multiclass/csvs/dataset21_stageB_valid.csv
wrote /kaggle/working/dataset21_classification/stageB_multiclass/csvs/dataset21_stageB_test.csv
wrote /kaggle/working/dataset21_classification/stageB_multiclass/csvs/dataset21_stageB_classes_onehot.csv


In [24]:
OUT = Path("/kaggle/working/dataset21_classification")
assert OUT.exists(), "dataset21_classification folder not found."

# creating one zip for the entire folder
zip_path = OUT.with_suffix(".zip")
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
    for root, _, files in os.walk(OUT):
        for f in files:
            fp = Path(root) / f
            z.write(fp, arcname=str(fp.relative_to(OUT.parent)))

print(zip_path)

/kaggle/working/dataset21_classification.zip
